In [ ]:
# mount google colab
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# install packages
! pip install transformers datasets
! pip install sentencepiece
! pip install transformers[torch]
! pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 6.2 MB/s eta 0:00:00


In [ ]:
# login to huggingface
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# import dataset wich should be used for finetuning
from datasets import load_dataset

left_dataset = load_dataset('csv', data_files='add_path_to_dataset/allsides_data_left_heading.csv')
left_dataset = left_dataset['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# split data into trainings and test split
left_dataset = left_dataset.train_test_split(test_size=0.2)

In [ ]:
left_dataset

Dataset({
    features: ['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'url', 'date', 'title', 'tags', 'heading', 'source', 'preview_text', 'full_text', 'bias_rating', 'full_text_cleaned'],
    num_rows: 11446
})

## Preprocess

In [ ]:
# import tokenizer for the pretrained model wich should be used
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-large")

In [ ]:
# function to apply tokenizing
def preprocess_function(examples):
    return tokenizer(examples["heading"]) #define column wich should be used for finetuning

In [ ]:
# tokenize dataset
tokenized_left_dataset = left_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=left_dataset["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/9156 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2290 [00:00<?, ? examples/s]

In [ ]:
# chunk size for concatenation
block_size = 32


# function to concatenate data
def group_texts(examples):
    # concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # split by chunks of block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
# apply concatenation to dataset
lm_dataset = tokenized_left_dataset.map(group_texts, batched=True, num_proc=4)
#lm_dataset['train'][0]

Map (num_proc=4):   0%|          | 0/9156 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2290 [00:00<?, ? examples/s]

In [ ]:
# import the Trainer and Datacollector for finetuning purposes
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

# add pad token to tokenizer
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) #load datacollector

## Train

In [ ]:
# import the pretrained model wich should be finetuned
from transformers import PegasusForCausalLM

model = PegasusForCausalLM.from_pretrained("google/pegasus-large", add_cross_attention=False)
print(model.config.is_decoder, f"{model.__class__} has to be configured as a decoder.")

Some weights of PegasusForCausalLM were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


True <class 'transformers.models.pegasus.modeling_pegasus.PegasusForCausalLM'> has to be configured as a decoder.


In [ ]:
#finetune the pretrained model on the dataset
training_args = TrainingArguments(
    output_dir="pegasus_left_heading_torch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    num_train_epochs=20,
    do_train=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

/content/pegasus_left_heading_torch is already a clone of https://huggingface.co/tobijen/pegasus_left_heading_torch. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,6.936113
2,No log,6.102550
3,No log,5.508801
4,No log,4.961810
5,No log,4.335215
6,No log,3.946341
7,No log,3.649539
8,No log,3.532418
9,No log,3.306130
10,No log,3.156908


TrainOutput(global_step=720, training_loss=4.317159610324436, metrics={'train_runtime': 159.5817, 'train_samples_per_second': 35.593, 'train_steps_per_second': 4.512, 'total_flos': 293086691328000.0, 'train_loss': 4.317159610324436, 'epoch': 20.0})

In [ ]:
# get the perplexity for the model
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 10.62


In [ ]:
# push finetuned model to huggingface repo
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/1.37G [00:00<?, ?B/s]

Upload file runs/Jul28_14-15-22_1bf844d39158/events.out.tfevents.1690553725.1bf844d39158.3220.2:   0%|        …

Upload file runs/Jul28_14-15-22_1bf844d39158/events.out.tfevents.1690553893.1bf844d39158.3220.3:   0%|        …

To https://huggingface.co/tobijen/pegasus_left_heading_torch
   a9b5287..6d2cd14  main -> main

   a9b5287..6d2cd14  main -> main

To https://huggingface.co/tobijen/pegasus_left_heading_torch
   6d2cd14..a06e628  main -> main

   6d2cd14..a06e628  main -> main



'https://huggingface.co/tobijen/pegasus_left_heading_torch/commit/6d2cd14a6c63eadc70e735a3c0e9019968f90825'

In [ ]:
# save model locally
trainer.save_model("path_to_where_the_model_should_be_saved_locally/pegasus_left_heading_torch")

## Inference

In [ ]:
# Inputs to generate text for
prompts = {
    "social_1": "Gay marriage is",
    "social_2": "Abortion is",
    "education_1": "Public education is",
    "education_2": "Charter schools are",
    "economic_1":"The increase of taxes is",
    "economic_2":"Government regulations on businesses are",
}

In [ ]:
from transformers import pipeline
# generate text with the pipeline module from the transformers library
generated_text_dict = {}
for key, prompt in prompts.items():
  print(key, " => ", prompt)
  generator = pipeline("text-generation", model="tobijen/pegasus_left_heading_torch", tokenizer=tokenizer)
  generated_text = generator(prompt, return_full_text=True, max_new_tokens=50)
  generated_text_dict[key] = generated_text[0]["generated_text"]
  print(generated_text)

social_1  =>  Gay marriage is


[{'generated_text': 'Gay marriage is crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis'}]
social_2  =>  Abortion is
[{'generated_text': 'Abortion is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is'}]
education_1  =>  Public education is
[{'generated_text': 'Public education is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is'}]
education_2  =>  Charter schools are
[{'generated_text': 'Charter schools are Charter Charter Charter Charter Charter Charter Charter Charter Charter Charter Charter Charter Charte

In [ ]:
generated_text_dict

{'social_1': 'Gay marriage is crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis crisis',
 'social_2': 'Abortion is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is',
 'education_1': 'Public education is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is',
 'education_2': 'Charter schools are Charter Charter Charter Charter Charter Charter Charter Charter Charter Charter Charter Charter Charter Charter Charter Charter Charter Charter Charter Charter Charter Charter Charter Charter Charter Charter Charter Charter Cha

In [ ]:
# add the generated text to a json file, wich is used for the evaluation
import json
import os

def write_dict_to_json_file(file_path, data):
    # Check if the JSON file exists
    if os.path.exists(file_path):
        # If the file exists, load the existing data
        with open(file_path, 'r', encoding='utf-8') as file:
            existing_data = json.load(file)
    else:
        # If the file does not exist, create an empty dictionary
        existing_data = {}

    # Update the existing dictionary with the new data
    existing_data.update(data)

    # Write the updated dictionary back to the JSON file
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False)

data_to_append = {
    "pegasus_left_heading": generated_text_dict
}

# write data to json file
write_dict_to_json_file('path_to_json_file/generated_texts.json', data_to_append)